In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.\
        builder.\
        config ("spark.sql.warehouse.dir","/user/hive/warehouse").\
        enableHiveSupport().\
        appName("Spark SQL - Data processing").\
        master("yarn").\
        getOrCreate()

In [3]:
spark

In [4]:
SparkContext

NameError: name 'SparkContext' is not defined

In [5]:
l = [("X", )]

In [6]:
df = spark.createDataFrame(l).toDF("dummy")

In [7]:
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



In [19]:
from pyspark.sql.functions import *

In [9]:
df.select(
    current_date().alias('current_date'), 
    year(current_date()).alias('year'),
    month(current_date()).alias('month'),
    weekofyear(current_date()).alias('weekofyear'),
    dayofyear(current_date()).alias('dayofyear'),
    dayofmonth(current_date()).alias('dayofmonth'),
    dayofweek(current_date()).alias('dayofweek')
).show() #yyyy-MM-dd

+------------+----+-----+----------+---------+----------+---------+
|current_date|year|month|weekofyear|dayofyear|dayofmonth|dayofweek|
+------------+----+-----+----------+---------+----------+---------+
|  2021-09-09|2021|    9|        36|      252|         9|        5|
+------------+----+-----+----------+---------+----------+---------+



In [10]:
df.select(
    current_timestamp().alias('current_timestamp'), 
    year(current_timestamp()).alias('year'),
    month(current_timestamp()).alias('month'),
    dayofmonth(current_timestamp()).alias('dayofmonth'),
    hour(current_timestamp()).alias('hour'),
    minute(current_timestamp()).alias('minute'),
    second(current_timestamp()).alias('second')
).show(truncate=False) #yyyy-MM-dd HH:mm:ss.SSS

+-----------------------+----+-----+----------+----+------+------+
|current_timestamp      |year|month|dayofmonth|hour|minute|second|
+-----------------------+----+-----+----------+----+------+------+
|2021-09-09 05:01:32.829|2021|9    |9         |5   |1     |32    |
+-----------------------+----+-----+----------+----+------+------+



In [11]:
datetimes = [(20140228, "28-Feb-2014 10:00:00.123"),
                     (20160229, "20-Feb-2016 08:08:08.999"),
                     (20171031, "31-Dec-2017 11:59:59.123"),
                     (20191130, "31-Aug-2019 00:00:00.000")
                ]

In [12]:
datetimesDF = spark.createDataFrame(datetimes, schema="date BIGINT, time STRING")

In [13]:
datetimesDF.show(truncate=False)

+--------+------------------------+
|date    |time                    |
+--------+------------------------+
|20140228|28-Feb-2014 10:00:00.123|
|20160229|20-Feb-2016 08:08:08.999|
|20171031|31-Dec-2017 11:59:59.123|
|20191130|31-Aug-2019 00:00:00.000|
+--------+------------------------+



In [14]:
df.select(to_date(lit('20210302'), 'yyyyMMdd').alias('to_date')).show()

+----------+
|   to_date|
+----------+
|2021-03-02|
+----------+



In [17]:
datetimesDF.show(truncate = False)

+--------+------------------------+
|date    |time                    |
+--------+------------------------+
|20140228|28-Feb-2014 10:00:00.123|
|20160229|20-Feb-2016 08:08:08.999|
|20171031|31-Dec-2017 11:59:59.123|
|20191130|31-Aug-2019 00:00:00.000|
+--------+------------------------+



In [42]:
datetimesDF. \
    withColumn('to_date', to_date(col('date').cast('string'), 'yyyyMMdd')). \
    withColumn('to_timestamp', to_timestamp(col('time'), 'dd-MMM-yyyy HH:mm:ss.SSS')). \
    show(truncate=False)

+--------+------------------------+----------+-----------------------+
|date    |time                    |to_date   |to_timestamp           |
+--------+------------------------+----------+-----------------------+
|20140228|28-Feb-2014 10:00:00.123|2014-02-28|2014-02-28 10:00:00.123|
|20160229|20-Feb-2016 08:08:08.999|2016-02-29|2016-02-20 08:08:08.999|
|20171031|31-Dec-2017 11:59:59.123|2017-10-31|2017-12-31 11:59:59.123|
|20191130|31-Aug-2019 00:00:00.000|2019-11-30|2019-08-31 00:00:00    |
+--------+------------------------+----------+-----------------------+



In [21]:
from pyspark.sql.functions import date_format

In [57]:
datetimesDF. \
     withColumn( "date_ym", date_format(to_date(col("date").cast('string') ,'yyyyMMdd'), 'yyyyMM')). \
     withColumn("time_ym",  col("time").cast('string')). \
     withColumn("time_ym1", to_timestamp(col('time'), 'dd-MMM-yyyy HH:mm:ss.SSS')). \
    show(truncate=False)

+--------+------------------------+-------+------------------------+-----------------------+
|date    |time                    |date_ym|time_ym                 |time_ym1               |
+--------+------------------------+-------+------------------------+-----------------------+
|20140228|28-Feb-2014 10:00:00.123|201402 |28-Feb-2014 10:00:00.123|2014-02-28 10:00:00.123|
|20160229|20-Feb-2016 08:08:08.999|201602 |20-Feb-2016 08:08:08.999|2016-02-20 08:08:08.999|
|20171031|31-Dec-2017 11:59:59.123|201710 |31-Dec-2017 11:59:59.123|2017-12-31 11:59:59.123|
|20191130|31-Aug-2019 00:00:00.000|201911 |31-Aug-2019 00:00:00.000|2019-08-31 00:00:00    |
+--------+------------------------+-------+------------------------+-----------------------+



In [58]:
datetimesDF. \
    withColumn('to_date', to_date(col('date').cast('string'), 'yyyyMMdd')). \
    withColumn('to_timestamp', to_timestamp(col('time'), 'dd-MMM-yyyy HH:mm:ss.SSS')). \
    withColumn('to_date_1', to_date(col('time'), 'yyyyMMdd')). \
    withColumn("time_ym", date_format("time", "yyyyMM").cast('int')). \
    show(truncate=False)

+--------+------------------------+----------+-----------------------+---------+-------+
|date    |time                    |to_date   |to_timestamp           |to_date_1|time_ym|
+--------+------------------------+----------+-----------------------+---------+-------+
|20140228|28-Feb-2014 10:00:00.123|2014-02-28|2014-02-28 10:00:00.123|null     |null   |
|20160229|20-Feb-2016 08:08:08.999|2016-02-29|2016-02-20 08:08:08.999|null     |null   |
|20171031|31-Dec-2017 11:59:59.123|2017-10-31|2017-12-31 11:59:59.123|null     |null   |
|20191130|31-Aug-2019 00:00:00.000|2019-11-30|2019-08-31 00:00:00    |null     |null   |
+--------+------------------------+----------+-----------------------+---------+-------+

